# **Validation Suite**

## Purpose and Setup
This notebook will allow you to test the output from WRF-GHG against surface, upper air, and satelite observations.

The next cell imports modules needed to work properly

In [ ]:
import wrf
import numpy as np
import pandas as pd
from datetime import datetime as dt
from netCDF4 import Dataset #type: ignore
from termcolor import cprint
from numpy import unravel_index

## *Base class*

This class is used for all the following classes in order to give some basic location data to each class.

In [ ]:
class Base_point:
    '''
    Base_point: parent class for validation. Sets location name.
    '''
    def __init__(self: object, loc: str, **kwargs) -> None:
        self.loc = loc

## *Upper air (UA) class*

This class is used to set the attributes for any data objects that cares about upper air data. This is usually the WRF data that you're testing or the radiosonde data you use for validation. It inherits from the base class.

In [ ]:
class UA_point(Base_point):
    def __init__(self: object, loc: str, **kwargs) -> None:
        super().__init__(loc, **kwargs)
    def __eq__(self: object, other: object) -> bool:
        try:
            results = np.empty(5, bool)
            results[0] = np.allclose(self.p, other.p, atol=10.)
            results[1] = np.allclose(self.t, other.t, atol=1.)
            results[2] = np.allclose(self.td, other.td, atol=1.)
            results[3] = np.allclose(self.wdir, other.wdir, atol=5.)
            results[4] = np.allclose(self.wspd, other.wspd, atol=1.)
            result = results.all()
        except AttributeError:
            if isinstance(self, WRF_point):
                result = other.__eq__(self)
            else:
                result = NotImplemented
        finally:
            return result

## *Satelite (Sat) class*

This class is used for the 